In [1]:
%matplotlib inline
from __future__ import print_function, division
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from os.path import join as pjoin
sns.set_style('darkgrid')

datapath = "C:/WorkSpace/data/Derived/impact/cairns/bysuburb"
filename = "cairns_aggimpact_classified_local_shield.csv"

data = pd.read_csv(pjoin(datapath, filename), header=0, skiprows=[1, 2])

Load the full output file, and do some initial tidying up (i.e. remove a bunch of the extraneous metadata fields). We also take the chance to calculate the structural loss for each permutation of the vulnerability models. This bit can take some time (!).

In [2]:
filename = "cairns_impact_classified_local_shield.csv"

data = pd.read_csv(pjoin(datapath, filename), header=0)
slr_cols = [col for col in data.keys() if col.startswith('structural_loss_ratio') and col[-1].isdigit()]
slrdf = data[slr_cols]
slrdf.loc[:, '0.2s gust at 10m height m/s'] = data['0.2s gust at 10m height m/s']
slrdf.loc[:, 'SUBURB_2015'] = data['SUBURB_2015']
slrdf.loc[:, 'LID'] = data['LID']
slrdf.loc[:, 'REPLACEMENT_VALUE'] = data['REPLACEMENT_VALUE']
slrdf.loc[:, 'M4'] = data['M4']
slrdf.loc[:, 'YEAR_BUILT'] = data['YEAR_BUILT']
sldf = slrdf[['0.2s gust at 10m height m/s', 'SUBURB_2015', 'LID', 'REPLACEMENT_VALUE', 'M4', 'YEAR_BUILT']]
sl_cols = []
for col in slr_cols:
    slcolname = col.replace("_ratio", "")
    sl_cols.append(slcolname)
    sldf.loc[:, slcolname] = slrdf[col] * sldf['REPLACEMENT_VALUE']

In [3]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 6))
meansl = np.mean(sldf.loc[:, sl_cols], axis=1).values
sumsl = np.sum(sldf.loc[:, sl_cols]).values
meanslmean = np.mean(meansl)
sumslmean = np.mean(sumsl)
sns.distplot(meansl, bins=np.arange(0, 600000, 30000),
             fit=stats.lognorm, axlabel="Mean structural loss ($)", ax=ax0, 
             kde_kws={'label':'KDE', 'gridsize':10000, 'bw':10000},
             hist_kws={'label':'Probability'},
             fit_kws={'color':'red', 'label':'Fitted lognormal'})
sns.distplot(sumsl, fit=stats.norm, axlabel="Total structural loss ($ million)", ax=ax1, 
             kde_kws={'label':'KDE'},
             hist_kws={'label':'Probability'},
             fit_kws={'color':'red', 'label':'Fitted normal'})
#ax0.axvline(meanslmean, label="Mean: ${0:0.0f}".format(meanslmean))
ax1.axvline(sumslmean, label="Mean: ${0:0.1f} million".format(sumslmean/10**6))

ax0.legend()
ax0.set_xlim((0, 600000))
ax0.set_yscale("log", nonposy='clip')
ax1.legend(loc=2)

meanslcov = np.std(meansl)/np.mean(meansl)
sumslcov = np.std(sumsl)/np.mean(sumsl)
print(meanslcov, sumslcov)

The left panel is the mean distribution of structural losses across the assets (y-scale is logarithmic). The greatest proportion of losses are less than \$10,000 (the mean is somewhere around \$10,300). The red curve is a fitted lognormal distribution. I've tried a number of other distributions (exponential, weibull, pareto) and none fit as well as the lognormal.

The right panel shows the distribution of total structural losses for the community. The red curve is a fitted normal distribution. 

In both cases, even though there's only 100 realisations of the losses, the distributions appear to be converging towards the respective fitted distributions. We can script this post-processing up and run a larger number of simulations (e.g. $10^5$) to confirm this convergence. 

In [4]:
pal = sns.color_palette("cubehelix",5)
sns.pairplot(data, vars=["0.2s gust at 10m height m/s", 
                         "structural_loss_ratio", 
                         "structural_loss", 
                         'M4', "REPLACEMENT_VALUE"],
             diag_kind='kde', diag_kws=dict(shade=True),  
             hue='YEAR_BUILT', palette=pal, size=2.5)

A scatter plot of the various continuous variables in the output (I've ignored contents value, since our wind vulneraility models don't consider contents). In the first instance, they're colour-coded based on the AS4055 site classification (i.e. slope,  terrain and shielding factors), which are linked to the address, not the building atttributes. 

Highest gust speeds are recorded in locations where M4 values are around 1.1.

Highest structural loss ratios occur at slightly lower M4 values, but near the peak gust wind speeds. The highest gusts don't produce the highest structural loss (or loss ratio). 

The loss vs loss ratio plot is curiously linear. I suspect that most of the data points are close to (0, 0), and we are seeing the extrema.

In [5]:
fig, axes = plt.subplots(1, 3, figsize=(16, 18), sharey=True)
ax = sns.stripplot(x='structural_loss_ratio', y='SUBURB_2015', data=data, jitter=True, ax=axes[0])
ax = sns.stripplot(x='structural_loss', y='SUBURB_2015', data=data, jitter=True, ax=axes[1])
ax = sns.stripplot(x='0.2s gust at 10m height m/s', y='SUBURB_2015', data=data, jitter=True, ax=axes[2])

axes[0].set_xlim((0,1))
axes[1].set_xlim((0, 600000))
fig.tight_layout()

In [6]:
slr = slrdf[slr_cols].values
slrm = np.mean(slr, axis=1)
sl = sldf[sl_cols].values
slm = np.mean(sl, axis=1)
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(12,16), sharex=True)
sns.regplot(np.where(slrdf['M4']==-9999, 1, slrdf['M4']), slrm, ax=ax0)
sns.regplot(sldf['M4'], slm, ax=ax1)
#plt.scatter(slrm, slrdf['M4'])
ax0.set_ylim((0, 1))
ax1.set_xlim((0.5, 1.5))
ax1.set_ylim((0, 600000))
ax0.set_ylabel("Mean structural loss ratio")
ax1.set_ylabel("Mean structural loss ($)")
ax1.set_xlabel("M4")

In [7]:
fig, axes = plt.subplots(1, 2,figsize=(16,8))
ax = sns.distplot(slm, fit=stats.lognorm, label='Mean loss per asset', kde_kws=dict(bw=5000, label="KDE"),
                  fit_kws={'color':'red', 'label':'Lognorm fit'}, ax=axes[0])
ax.set_xlim((0, 600000))
ax.set_yscale("log", nonposy='clip')
ax.set_xlabel("Mean structural loss ($)")
ax.legend()

ax = sns.distplot(slrm, fit=stats.lognorm, label='Mean loss ratio', kde_kws=dict(bw=0.025, label="KDE"),
                  fit_kws={'color':'red', 'label':'Lognorm fit'}, ax=axes[1])
ax.set_xlim((0, 1))
ax.set_yscale("log", nonposy='clip')
ax.set_xlabel("Mean loss ratio")
ax.legend()

In [8]:
suburb_sl = sldf.groupby('SUBURB_2015')
suburb_sl.agg([sum, np.mean, np.std, np.max])

In [9]:
sns.jointplot('structural_loss_ratio', 'structural_loss', data, kind='kde', xlim=(0,0.2), ylim=(0,50000) )

In [10]:
pd.pivot_table(data, index='SUBURB_2015', columns='AS4055_CLASS', values='structural_loss_ratio', aggfunc=np.max)

In [11]:
sns.distplot(data['structural_loss_ratio'])